<a href="https://colab.research.google.com/github/maksgorbachev/mipt_lab/blob/master/%D0%92%D1%8B%D1%87%D0%B8%D1%81%D0%BB%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B0_%D0%9B_%D1%80_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Проверка решения методом Гаусса

A = np.array ([
  [10, 10, 10],
  [ 1,  91,  4],
  [ 7,  8,  9]
], dtype=float)

x = np.array ([10, 2, 3]) # Готовим результат
b = A.dot (x)             # Зная результат, вычисляем b

# Посмотрим как выглядит наша матрица
Ab = np.column_stack ([A, b])
print (Ab)

# Решаем и сравниваем результат
result_x = solveSLAE_Gauss (A, b)

print (f"x: {x}")
print (f"solve: {result_x}")
print (f"The result is " + int(isZeroArr (x - result_x) == False) * "in" + "correct!")

[[ 10.  10.  10. 150.]
 [  1.  91.   4. 204.]
 [  7.   8.   9. 113.]]
x: [10  2  3]
solve: [10.  2.  3.]
The result is correct!


## Метод Якоби

In [4]:
def checkSizeSLAE (A, b):
  if (A.shape[0] != A.shape[1]):
    raise Exception ("A is not square matrix!")

  if (A.shape[0] != b.size):
    raise Exception ("B has incorrect size!")

def checkOnDiag (A):
  size = A.size
  for i in range (size):
    sum = np.sum (np.abs (A[i])) - 2 * np.abs (A[i][i])
    if (sum < 0):
      return True
  
  return False

# Условие остановки
def solverSLAE_Jacobi (A, b, steps):
  checkSizeSLAE (A, b)
  if (checkOnDiag (A) == False):
    raise Exception ("Нет диагонального преобладания!")

  size = b.size;
  x = np.zeros (size);

  for step in range (steps):
    x_cur = x.copy ()
    for i in range (size):
      a = A[i][i]
      temp = np.sum (A[i] * x_cur) - x_cur[i] * a
      x[i] = (b[i] - temp) / a
  
  return x

result_x = solverSLAE_Jacobi (A, b, 250)

print (f"x: {x}")
print (f"solve: {result_x}")
print (f"The result is " + int(isZeroArr (x - result_x) == False) * "in" + "correct!")

x: [10  2  3]
solve: [9.99999954 1.99999998 2.9999996 ]
The result is correct!


## Метод Зейделя

In [5]:
def solverSLAE_Seidel (A, b, steps):
  checkSizeSLAE (A, b)

  size = b.size;
  x = np.zeros (size);

  for step in range (steps):
    for i in range (size):
      a = A[i][i]
      temp = np.sum (A[i] * x) - x[i] * a
      x[i] = (b[i] - temp) / a
  
  return x

result_x = solverSLAE_Seidel (A, b, 65)

print (f"x: {x}")
print (f"solve: {result_x}")
print (f"The result is " + int(isZeroArr (x - result_x) == False) * "in" + "correct!")

x: [10  2  3]
solve: [10.00000038  2.00000001  2.9999997 ]
The result is correct!


In [6]:
import numpy as np

_zero_eps = 1e-6

def isZero (number):
  return abs (number) < _zero_eps

def isZeroArr (numArr):
  return np.sum ([isZero (num) for num in numArr]) == numArr.size

def triangalizeMatrix (A):
  if ((A.shape[0] <= A.shape[1]) == False):
    raise Exception ("A is not square matrix!")

  size = A.shape[0]

  for ref_col in range (size - 1):

    if isZero (A[ref_col, ref_col]):
      # Находим первый ненулевой элемент в столбце
      non_zero_row = -1
      for row in range (ref_col, size):
        if isZero (A[row, ref_col]) == False:
          non_zero_row = row
          break;

      if (non_zero_row == -1):
        continue

      A[ref_col] = A[ref_col] - A[non_zero_row]

    ref_row = ref_col

    A_ref = A[ref_row, ref_col]

    for row in range (ref_row + 1, size):
      if (isZero(A[row, ref_col])):
        continue

      alpha = A_ref / A[row, ref_col]

      A[row] = A[row] * alpha- A[ref_col]

  return A

# Example:
# Ax = b, then
#   [in]:  Ab
#   [out]: x
# Only for matrix [n, n + 1]
def solveTriangMatrix (A):
  num_rows = A.shape[0]
  num_cols = A.shape[1]

  if num_cols - num_rows != 1:
    raise Exception (f"Incorrect size of matrix! Size: [{num_rows}, {num_cols}]")

  x = np.zeros (num_cols)
  x[num_cols - 1] = -1

  for row in range (num_rows - 1, -1, -1):
    x[row] = -np.sum (A[row].dot (x)) / A[row, row]

  return x[:-1]

# Ax = b
# A - square matrix
# Where Rg(A) = dim (A)
def solveSLAE_Gauss (A, b):
  Ab = np.column_stack ([A, b])

  Ab_Triang = triangalizeMatrix (Ab)
  x = solveTriangMatrix (Ab_Triang)

  return x


## Метод Гаусса для 3-ёх диагональной матрицы

In [ ]:
def solve_3DiagSLAE_Gauss (A, b):
  checkSizeSLAE (A, b)

  size = b.size
  C = np.zeros ([size, 3])

  # Прямая прогонка
  C[0] = [A[0][0], A[0][1], b[0]]
  
  for i in range (1, size - 1):
    k = C[i-1][0]
    C[i] = [
      A[i][i] - A[i][i-1]*C[i-1][1]/k,
      A[i][i+1],
      b[i] - A[i][i-1]*C[i-1][2]/k
    ]

  i = size - 1
  k = C[i-1][0]
  C[i] = [
      A[i][i] - A[i][i-1]*C[i-1][1]/k,
      0,
      b[i] - A[i][i-1]*C[i-1][2]/k
    ]

  print (C)

  # Обратная прогонка
  x = np.zeros (size)
  
  x[size-1] = C[size-1][2] / C[size-1][0]

  for i in range (size - 2, -1, -1):
    x[i] = (C[i][2] - C[i][1] * x[i+1]) / C[i][0]

  return x


# Testing this method

A = np.array ([
  [1, 2,  0,  0,  0],
  [1, 6,  3,  0,  0],
  [0, 2,  4, -5,  0],
  [0, 0, -1,  7,  4],
  [0, 0,  0,  3, -6],
], dtype=float)

x = np.array ([1, 2, 3, 4, 5])
b = A.dot (x)

result_x = solve_3DiagSLAE_Gauss (A, b)

print (f"x: {x}")
print (f"solve: {result_x}")
print (f"The result is " + int(isZeroArr (x - result_x) == False) * "in" + "correct!")

[[  1.    2.    5. ]
 [  4.    3.   17. ]
 [  2.5  -5.  -12.5]
 [  5.    4.   40. ]
 [ -8.4   0.  -42. ]]
x: [1 2 3 4 5]
solve: [1. 2. 3. 4. 5.]
The result is correct!


## Метод Невязок

In [ ]:
# Критерий остановки
def solveSLAE_Residual_1 (A, b, steps):
  checkSizeSLAE (A, b)
  
  size = b.size
  x = np.zeros (size)

  for step in range (steps):
    r = A.dot (x) - b
    t = r * r / (A.dot (r) * r)
    x_old = x
    x = x - t * r

    if (np.max (np.abs (x - x_old)) < _zero_eps):
      print (f"step: {step}")
      return x

  return x

steps = 200

result_x = solveSLAE_Residual_1 (A, b, steps)
eps_residual_1 = np.max (np.abs (x - result_x))

print (f"x: {x}")
print (f"solve: {result_x}")
print (f"The result is " + int(isZeroArr (x - result_x) == False) * "in" + "correct!")
print (f"eps: {eps_residual_1}")

step: 172
x: [1 2 3 4 5]
solve: [1.0000001  2.00000016 2.99999976 4.00000046 4.99999961]
The result is correct!
eps: 4.6383877450040245e-07


In [ ]:
def solveSLAE_Residual_2 (A, b, steps):
  checkSizeSLAE (A, b)
  
  size = b.size
  x = np.zeros (size)

  for step in range (steps):
    r = A.dot (x) - b
    t = r * A.dot (r) / (A.dot (r) * A.dot (r))
    x = x - t * r

  return x

result_x = solveSLAE_Residual_2 (A, b, steps)
eps_residual_2 = np.max (np.abs (x - result_x))

print (f"x: {x}")
print (f"solve: {result_x}")
print (f"The result is " + int(isZeroArr (x - result_x) == False) * "in" + "correct!")
print (f"eps: {eps_residual_2}")

print (f"Отношение погрешностей: {eps_residual_1 / eps_residual_2}")


x: [1 2 3 4 5]
solve: [-1.18381793  3.90136533  4.19928058  7.11505642  9.09679004]
The result is incorrect!
eps: 4.096790041477361
Отношение погрешностей: 1.0
